In [1]:
import pandas as pd 
import json
from glob import glob

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
def inspect(inspect_idx, path="data/finetune-03/*.json"):
    dfs = {}
    jsons = glob(path)
    results = {0:[], inspect_idx:[]}
    for path in jsons:
        name = path.split("/")[-1].split("_")[0]
        with open(path, "rt") as f:
            y = json.load(f)
            for iterations in [0, inspect_idx]:
                scores = y["history"][iterations]["scores"]
                scores["name"] = name
                results[iterations].append(scores)
    for k,v in results.items():
        dfs[k] = pd.DataFrame(v)
    print("count")
    print(len(dfs[0]))
    print("")
    print("improvement")
    print(dfs[inspect_idx].count() - dfs[0].count())
    print("")
    print("by_name")
    print(dfs[inspect_idx].groupby("name").count())
    # print(dfs)

In [3]:
inspect(1, path="data/finetune-03/*.json")

count
570

improvement
lex                         0
parse                       0
type_check                232
build                      63
position                   44
normalize                  40
position_concentration     40
pnl                        40
mdd                        40
ret                        40
std                        40
tvr                        40
max_tvr                    40
name                        0
dtype: int64

by_name
               lex  parse  type_check  build  position  normalize  position_concentration  pnl  mdd  ret  std  tvr  max_tvr
name                                                                                                                       
adl             11     11           6      0         0          0                       0    0    0    0    0    0        0
adx             16     16          12      0         0          0                       0    0    0    0    0    0        0
aroon           19     19           6  

In [7]:
# !cat `ls -rt ./data/finetune-03/adx_*.json | tail -n1` | sed -e 's/\\n/\n/g' 
!cat `ls -rt ./data/finetune-03/adx_*.json  | tail -n1` 

{
    "desired_response": "# True Range (TR)\ntr(high: Signal<Float>, low: Signal<Float>, close: Signal<Float>) : Signal<Float> = {\n    # Measures directional movements, considering price gaps and daily volatility. \n    close_d1 = ts_delay(signal=close, period=1)\n    high_low   : Signal<Float> = subtract(x=high, y=low)\n    high_close : Signal<Float> = abs(subtract(x=high, y=close_d1))\n    low_close  : Signal<Float> = abs(subtract(x=low, y=close_d1))\n    result     : Signal<Float> = max(x=high_low, y=max(x=high_close, y=low_close))\n}\n\n# Averaged True Range (ATR)\natr(high: Signal<Float>, low: Signal<Float>, close: Signal<Float>, period: Int) : Signal<Float> = {\n    result : Signal<Float> = ts_mean(signal=tr(high=high, low=low, close=close), period=period)\n}\n\n# Directional Movement (+DI, -DI)\ndi_plus(high: Signal<Float>, low: Signal<Float>, close: Signal<Float>, period: Int) : Signal<Float> = {\n    up_move : Signal<Float> = ts_diff(signal=high, period=1)\n    plus_dm : Sig

In [9]:
!cat `ls -rt ./data/finetune-02/*.json | tail -n1` | sed -e 's/\\n/\n/g' 

{
    "model": "gpt-oss:120b",
    "created_at": "2026-01-12T08:36:57.34067157Z",
    "message": {
        "role": "assistant",
        "content": "{
  \"thought\": \"The goal is to write a ButterFlow script that computes the typical price (average of high, low, close), the true range, and the Keltner Channel bands (upper and lower). The script will load high, low, and close price series using the data intrinsic, define configurable parameters for the ATR window, ATR multiplier, and EMA window, implement helper functions for typical price and true range using intrinsic operators (add, divide, subtract, abs, max, multiply, ts_delay), calculate the ATR and EMA (middle line) with exponential decay functions, and finally compute the upper and lower bands. The script ends with an assignment to the special variable `result`, as required.\",
  \"code\": \"high : Signal<Float> = data(id=\\\"HIGH\\\")\
low : Signal<Float> = data(id=\\\"LOW\\\")\
close : Signal<Float> = data(id=\\\"CLOSE\\\")\
\

In [8]:
!cat `ls -rt ./data/finetune-03/adx_*.json | tail -n1` | sed -e 's/\\n/\n/g' 

{
    "desired_response": "# True Range (TR)
tr(high: Signal<Float>, low: Signal<Float>, close: Signal<Float>) : Signal<Float> = {
    # Measures directional movements, considering price gaps and daily volatility. 
    close_d1 = ts_delay(signal=close, period=1)
    high_low   : Signal<Float> = subtract(x=high, y=low)
    high_close : Signal<Float> = abs(subtract(x=high, y=close_d1))
    low_close  : Signal<Float> = abs(subtract(x=low, y=close_d1))
    result     : Signal<Float> = max(x=high_low, y=max(x=high_close, y=low_close))
}

# Averaged True Range (ATR)
atr(high: Signal<Float>, low: Signal<Float>, close: Signal<Float>, period: Int) : Signal<Float> = {
    result : Signal<Float> = ts_mean(signal=tr(high=high, low=low, close=close), period=period)
}

# Directional Movement (+DI, -DI)
di_plus(high: Signal<Float>, low: Signal<Float>, close: Signal<Float>, period: Int) : Signal<Float> = {
    up_move : Signal<Float> = ts_diff(signal=high, period=1)
    plus_dm : Signal<Float> = where

In [5]:
# inspect(1, path="data/finetune-03-2/*.json")